In [1]:
! python3 -m pip install nbformat


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go

import datetime

from sklearn.preprocessing import MinMaxScaler
from numpy.fft import fft
# from statsmodels.graphics.tsaplots import plot_acf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape

from holt_winters import HoltWinters

In [3]:
pjme = pd.read_csv('PJME_hourly.csv',
                   index_col=[0], parse_dates=[0]).sort_index().rename(columns={"PJME_MW": "mw"})
# pjme.head(5)

In [4]:
pjme['id'] = "pjme"
pjme['date'] = pjme.index
pjme = pjme[300:700]
y = pd.Series(MinMaxScaler().fit_transform(
    pd.DataFrame(pjme[["mw"]]))[:, 0], index=pjme.index)
df = pd.DataFrame({"x": list(range(len(y))), "y": y})

In [5]:
px.line(x=df.x, y=df.y)

### Входные параметры программы

In [6]:
from dataclasses import dataclass


@dataclass
class InputParams:
    alpha1 = 0.4
    alpha2 = 0.6
    alpha3 = 0.2
    period_length_seasons = 24  # длина периода в сезонах
    prediction_horizon = 2

In [7]:
input_params = InputParams()

### Поиск a(0), b(0)

In [8]:
lin_reg = LinearRegression()

lin_reg = lin_reg.fit(df[["x"]], df[["y"]])

In [9]:
a0, b0 = lin_reg.coef_[0][0], lin_reg.intercept_[0]
a0, b0

(np.float64(-0.0005953813862412849), np.float64(0.693768106014528))

In [10]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df.x,
        y=df.y, name="original time series")
)
fig.add_trace(
    go.Scatter(
        x=df.x,
        y=lin_reg.predict(df[["x"]])[:, 0], name="Fitted linear regression")
)
fig.update_layout(title="Поиск значений a(0), b(0) для инициализации")

In [11]:
hw = HoltWinters(df.y.to_list(),
                 slen=input_params.period_length_seasons,
                 alpha=input_params.alpha1,
                 beta=input_params.alpha2,
                 gamma=input_params.alpha3,
                 n_preds=input_params.period_length_seasons*input_params.prediction_horizon)

In [12]:
pred = hw.triple_exponential_smoothing(a0,b0)
df_pred = pd.DataFrame({"x": list(range(len(pred))),
                        "y": pred})
len(df_pred)

a0 =-0.0005953813862412849
b0 =0.693768106014528


448

In [13]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df.x,
        y=df.y, name="original time series"))
fig.add_trace(
    go.Scatter(
        x=df_pred.x,
        y=df_pred.y, name="Predicted time series"))

fig.update_layout(title="Предсказание модели")
